<div style="padding-bottom:30px">
<a href="https://github.com/iqtk/iqtk"><img src="https://raw.githubusercontent.com/iqtk/iqtk/master/inquiry/docs/assets/logotype_blue_small.png" style="width:100px; margin-left:0px"></img></a>
<p style="color:#9E9E9E">
<a href="https://github.com/cwbeitel/inquiry/tree/master/docs">Getting Started Guide</a> // <a href="https://goo.gl/forms/2cOmuUrQ3n3CKpim1">Documentation Feedback</a></p>
</div>

<h1 style="color:#9E9E9E">Metabolomics data preprocessing</h1>

In this tutorial we initiate a distributed file conversion of Agilent .d files into .mzML from the DataFlow UI permitting downstream analysis with XCMS3 or alternative tools.

To review, here we'll be running workflows generated by the Inquiry Toolkit directly from the Cloud DataFlow UI so once you've completed the [getting started documentation]() you'll be ready for this tutorial!

<h2 style="color:#9E9E9E">Configuration and Run</h2>

The first thing we need to do is parameterize our analysis. The parameters file we'll need to provide should have the following structure.

```json
{
  "archives": [
    "gs://inquiry-test/tests/toolkit-ms/file.d"
   ]
}

```

(Dataflow submission UI screenshot)

Once our job is submitted we'll be able to check its status using the Cloud DataFow UI as described in the [getting started documentation](). For illustration this workflow should look like the following:

(Dataflow job running screenshot)

When the workflow is complete you can obtain the path in Google Cloud Storage to the resulting files as described in the [getting started documentation]().

<h2 style="color:#9E9E9E">Exploring the data</h2>

The above workflow results in converted mzML files ready to be used with XCMS3. What follows is a tutorial on obtaining and performing basic analysis of a whole metabolome dataset from BigQuery which was not desposited by the above workflow. First we obtain the data:

In [3]:
import google.datalab.bigquery as bq
query = bq.Query('SELECT * FROM `jbei-cloud.metabolomics.feihnexample3` LIMIT 3')
output_options = bq.QueryOutput.table(use_cache=False)
result = query.execute(output_options=output_options).result()
result.to_dataframe()

,Samples,Factors,_1_2_4_benzenetriol,_1_monostearin,_2_hydroxyvaleric_acid,_3_phosphoglycerate,_5_hydroxynorvaline_NIST,adenosine,adenosine_5_monophosphate,adipic_acid,...,_222169,_222058,_222051,_222047,_222036,_222029,_199553,_199322,_207188,_199232
0,LabF_115909,Arabidopsis Genotype:fatb-ko KD; At1g08510 | P...,3566,450,931,2005,2403,407,1650,8148,...,254,358,788,2015,7857,1496,378,1440,916,324
1,LabF_115904,Arabidopsis Genotype:fatb-ko KD; At1g08510 | P...,1874,987,771,2039,2297,538,2153,2197,...,254,602,949,1952,5013,1840,443,1498,980,321
2,LabF_115914,Arabidopsis Genotype:fatb-ko KD; At1g08510 | P...,1945,1910,1114,1496,2598,575,2204,2149,...,651,751,721,919,5963,1870,425,1423,1006,228


<h3 style="color:#9E9E9E">References</h3>

1. Trapnell, Cole, et al. "Differential gene and transcript expression analysis of RNA-seq experiments with TopHat and Cufflinks." Nature protocols 7.3 (2012): 562-578.
2. Trapnell, Cole, Lior Pachter, and Steven L. Salzberg. "TopHat: discovering splice junctions with RNA-Seq." Bioinformatics 25.9 (2009): 1105-1111.
3. Langmead, Ben, and Steven L. Salzberg. "Fast gapped-read alignment with Bowtie 2." Nature methods 9.4 (2012): 357-359.

<h3 style="color:#9E9E9E">Contact</h3>

Want to get in touch? You can [provide feedback](https://goo.gl/forms/2cOmuUrQ3n3CKpim1) regarding this or other documentation,
[reach out to us](https://goo.gl/forms/j8FWdNJqABAoJvcW2) regarding collaboration, or [request a new feature or analytical capability](https://goo.gl/forms/dQm3SDcoNZsV7AAd2). We're looking forward to hearing from you!

<div style="padding-top: 30px">
<p style="color:#9E9E9E; text-align:center">This notebook was prepared for <a href="https://github.com/cwbeitel/inquiry">Project Inquiry</a> in support of the research mission of the Joint BioEnergy Institute (JBEI). Learn more at https://www.jbei.org/.</p>
<p style="color:#9E9E9E; text-align:center">The Joint BioEnergy Institute is a program of the U.S. Department of Energy Office of Science.</p>
<p style="color:#9E9E9E; text-align:center">© Regents of the University of California, 2017. Licensed under a BSD-3 <a href="https://github.com/cwbeitel/inquiry/blob/master/LICENSE">license</a>.</p>
<img src="https://raw.githubusercontent.com/iqtk/iqtk/master/inquiry/docs/assets/logotype_blue_small.png" style="width:100px"></img>
</div>